#**FINE TUNING NEURAL NETWORK HYPERPARAMETERS**

In [ ]:
%pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.0 MB/s eta 0:00:00


In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(x_train_full, y_train_full), (x_test, y_test) = fashion_mnist
x_train, y_train = x_train_full[:-5000], y_train_full[:-5000]
x_valid, y_valid = x_train_full[-5000:], y_train_full[-5000:]

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
x_train, x_test, x_valid = x_train/255., x_test/255., x_valid/255.

In [ ]:
import keras_tuner as kt

def model_build(hyper_parameters):
  n_hidden = hyper_parameters.Int("n_hidden", min_value=0, max_value=8, default=2)
  n_neurons = hyper_parameters.Int("n_neurons", min_value=16, max_value=256)
  learning_rate = hyper_parameters.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")

  optimizer = hyper_parameters.Choice("optimizers", values=["sgd", "adam"])
  if optimizer == "sgd":
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
  else:
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten())
  for _ in range(n_hidden):
    model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

  model.add(tf.keras.layers.Dense(10, activation="softmax"))
  model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

  return model

In [ ]:
random_search_tuner = kt.RandomSearch(
    model_build, objective="val_accuracy", max_trials=5, overwrite=True, directory="my_fashion_mnist", project_name="my_rnd_search", seed=42)

random_search_tuner.search(x_train, y_train, epochs=10, validation_data=(x_valid, y_valid))

Trial 5 Complete [00h 01m 23s]
val_accuracy: 0.7056000232696533

Best val_accuracy So Far: 0.8629999756813049
Total elapsed time: 00h 06m 05s


In [ ]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

In [ ]:
top3_parameters = random_search_tuner.get_best_hyperparameters(num_trials=3)
for i in range(3):
  print(top3_parameters[i].values)
  print()

{'n_hidden': 8, 'n_neurons': 37, 'learning_rate': 0.008547485565344062, 'optimizers': 'sgd'}

{'n_hidden': 4, 'n_neurons': 74, 'learning_rate': 0.00905127409782462, 'optimizers': 'adam'}

{'n_hidden': 7, 'n_neurons': 100, 'learning_rate': 0.0012482904754698163, 'optimizers': 'sgd'}



In [ ]:
best_trials = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trials.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
n_neurons: 37
learning_rate: 0.008547485565344062
optimizers: sgd
Score: 0.8629999756813049


In [ ]:
best_trials.metrics.get_last_value("val_accuracy")

0.8629999756813049

If we want to fine-tune the preprocessing part then we can split the model_build function into two parts:

1. Make a function called as build for doing the same objective as model_build.
2. And second function called fit (say) takes hyperparamteres object and a xompiled moedl as an argument, as well as all the model.fit() arguments, and fits the model and returns the history object.

In [ ]:
class MyclassificationHperModel(kt.HyperModel):
  def build(self, hyper_parameters):
    return model_build(hyper_parameters)

  def fit_model(self, hyper_parameters, model, x, y, **kwargs):
    if hyper_parameters.Boolean("normalize"):
      normalized_layer = tf.keras.layers.Normalization()
      x = normalized_layer(x)
    return model.fit(x, y, **kwargs)

In [ ]:
hyperband_tuner = kt.Hyperband(
    MyclassificationHperModel(), objective="val_accuracy", seed=42, max_epochs=10, factor=3, hyperband_iterations=2, overwrite=True, directory="my_fashion_mnist", project_name="hyperband"
)

In [ ]:
%pip install -q -U tensorboard-plugin-profile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.1 MB/s eta 0:00:00


In [ ]:
from pathlib import Path

In [ ]:
root_logdir = Path(hyperband_tuner.project_dir)/"tensorboard"
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2)
hyperband_tuner.search(x_train, y_train, epochs=10, validation_data=(x_valid, y_valid), callbacks=[early_stopping_cb, tensorboard_cb])

Trial 60 Complete [00h 00m 44s]
val_accuracy: 0.8644000291824341

Best val_accuracy So Far: 0.8906000256538391
Total elapsed time: 00h 43m 36s


In [ ]:
bayesian_optimizer_tuner = kt.BayesianOptimization(
    MyclassificationHperModel(), objective="val_accuracy", seed=42, max_trials=10, alpha=1e-4, beta=2.6, overwrite=True, directory="My_fashion_mnist", project_name="bayesian_opt")
bayesian_optimizer_tuner.search(x_train, y_train, epochs=10, validation_data=(x_valid, y_valid), callbacks=[early_stopping_cb, tensorboard_cb])

Trial 10 Complete [00h 01m 30s]
val_accuracy: 0.8805999755859375

Best val_accuracy So Far: 0.8808000087738037
Total elapsed time: 00h 15m 52s
